Imports and data preparation

In [73]:
%matplotlib inline
import pandas as pd
import quandl , math ,datetime
import numpy as np
from sklearn import preprocessing , model_selection , svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt

from matplotlib import style

style.use('ggplot')


df = pd.read_csv("/home/balerion/Desktop/Ai/GOOGL.csv")

# print(df.head())

df = df[["Open" , "High" , "Low" , "Close" , "Volume"]]

# df["HL_PCT"] = (df["High"] - df["Low"]) / df["Low"] * 100

df["HC_PCT"] = (df["High"] - df["Close"]) / df["Close"] * 100.0

df["PCT_change"] = (df["Close"] - df["Open"]) / df["Open"] * 100.0

df = df[["Close" , "HC_PCT" , "PCT_change" , "Volume"]]

# print(df.head())


forecast_col = "Close"

df.fillna(-99999 , inplace=True) # to get rid of nill data

forecast_out = int(math.ceil(0.01*len(df))) # for every 40 odd days

print(forecast_out)

df["label"] = df[forecast_col].shift(-forecast_out)

# print(df.head())



45


Prprocessing and Scaling

In [74]:
x = np.array(df.drop(columns=["label"]))# x = np.array(df.drop(columns = ["label"])) -> contains features
x = preprocessing.scale(x)
x = x[:-forecast_out]

x_lately = x[-forecast_out:]

# standardization of data x' = x - mean / standard deviation
# Standardized data refers to data that has been transformed to have a mean of 0 and a standard deviation of 1
# x = x[:-forecast_out+1]

df.dropna(inplace=True)
y = np.array(df["label"])
y = np.array(df["label"])

print(len(x) , len(y))



4386 4386


Why Scaling is important ?


In [12]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# # Sample DataFrame
# data = {
#     'feature1': [1, 2, 3, 4, 5],
#     'feature2': [10, 20, 30, 40, 50],
#     'feature3': [100, 200, 300, 400, 500]
# }
# df = pd.DataFrame(data)

# # Extract features
# x = df.values

# # Standardize the features
# scaler = StandardScaler()
# x_standardized = scaler.fit_transform(x)

# print("Original data:\n", x)
# print("Standardized data:\n", x_standardized)


#  Q. Why Standardization is Important ?

#  -> Improved Algorithm Performance:
# Many machine learning algorithms, such as those using gradient descent (e.g., linear regression, logistic regression) or distance metrics (e.g., k-nearest neighbors, support vector machines), perform better or converge faster with standardized data.

#  -> Fair Feature Contribution: 
# Standardization ensures that each feature contributes equally to the model. Without standardization, features with larger ranges can dominate the learning process, leading to biased results.

#  -> Stable and Faster Convergence: 
# For optimization algorithms, such as gradient descent, standardized data ensures stable and faster convergence by providing a smoother cost function landscape.

Model training and testing

In [75]:
x_train , x_test , y_train , y_test = model_selection.train_test_split(x , y , test_size=0.2)
classifier =  LinearRegression(n_jobs=10)   #svm.SVR(kernel = "poly")

classifier.fit(x_train , y_train)  # fit("used for training the model")

accuracy = classifier.score(x_test , y_test) # score("used for testing accuracy")

forecast_set = classifier.predict(x_lately)

print(forecast_set , accuracy , forecast_out)

# print(accuracy)


[3075.4663484  3063.13022392 3065.60959038 3107.73586966 3087.97538369
 3035.86375894 3019.2642464  3025.64928733 2947.33047419 3018.79791446
 2947.37583617 2934.77132493 2967.171428   2944.7700348  2965.30929664
 3052.60634043 3068.44352099 3061.84916503 3064.82707408 3022.13496764
 2979.49114388 3039.42660062 2996.78437246 2939.89158277 2935.06772744
 2974.19251664 3039.13706255 3047.21624675 3064.20981492 3038.63747848
 3038.08846384 3029.02340756 3002.86536453 3003.65498202 2994.99986941
 2866.09381011 2863.18767646 2838.62400404 2879.92333792 2898.37086526
 2932.76036168 2880.74142159 2899.23733002 2818.66790682 2806.19820628] 0.9846521531617511 45


What is SVR ?

In [59]:
# Key Concepts of SVR

# Margin of Tolerance : SVR introduces a margin of tolerance, within which errors are ignored. The goal is to find a function that deviates from the actual data points by a value no greater than tolerance.

# Support Vectors: Data points that lie within the margin boundaries and influence the position and orientation of the regression line.

# Kernel Trick: SVR can use kernel functions (like linear, polynomial, or RBF) to handle non-linear relationships by mapping input features into higher-dimensional spaces.